
## Data set description
The simulated data set comprises:

`room_id`: Unique identifier for each room (e.g., R001, R002).

`temperature`: Current temperature reading from the sensor (in °C).

`humidity`: Current humidity level reading from the sensor (in %).

`timestamp`: Time when the reading was recorded (automatically generated by Spark).
The data is generated at a rate of 5 rows per second, simulating multiple rooms with various environmental conditions.


## Challenges
Monitoring indoor environmental conditions poses several challenges:

**High data velocity**: Continuous data from multiple sensors can overwhelm traditional systems.

**Need for immediate alerts**: Delays in identifying critical conditions can lead to discomfort or system inefficiencies.

**Need for data aggregation and analysis**: Efficiently aggregating and analyzing real-time data for proactive maintenance and optimization is essential.

## Apache Spark with structured streaming
To address these challenges, Apache Spark is employed for its powerful distributed computing capabilities, enabling real-time data processing and analytics.


In [1]:
!pip install pyspark==3.1.2 -q
!pip install findspark -q

In [2]:
# You can also use this section to suppress warnings generated by your code:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

# FindSpark simplifies the process of using Apache Spark with Python

import findspark
findspark.init()

#import functions/Classes for sparkml

from pyspark.ml.clustering import KMeans


from pyspark.sql import SparkSession


### Set up the Spark session:


In [3]:
from pyspark.sql import SparkSession

# Initialize Spark Session
spark = SparkSession.builder \
    .appName("Smart Building HVAC Monitoring") \
    .getOrCreate()


24/10/24 19:00:24 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


### Simulate sensor data:

Use Spark’s rate source to generate continuous readings from multiple rooms.


In [4]:
from pyspark.sql.functions import expr, rand,when

# Simulate sensor data with room IDs and readings
sensor_data = spark.readStream.format("rate").option("rowsPerSecond", 5).load() \
    .withColumn("room_id", expr("CAST(value % 10 AS STRING)")) \
    .withColumn("temperature", when(expr("value % 10 == 0"), 15)  # Set temperature to 15 for one specific record
                .otherwise(20 + rand() * 25)) \
    .withColumn("humidity", expr("40 + rand() * 30"))

### Create a temporary SQL view:

Create temporary SQL view to perform SQL queries on the streaming data.


In [5]:
# Create a temporary SQL view for the sensor data
sensor_data.createOrReplaceTempView("sensor_table")


### Define SQL queries for aggregation and analysis:

* **Critical temperature query**: Detect rooms with critical temperature levels
* **Average readings query**: Calculate average readings over a 1-minute window
* **Attention needed query**: Identify rooms that need immediate attention based on humidity levels


In [6]:
# SQL Query to detect rooms with critical temperatures
critical_temperature_query = """
    SELECT 
        room_id, 
        temperature, 
        humidity, 
        timestamp 
    FROM sensor_table 
    WHERE temperature < 18 OR temperature > 60
"""

# SQL Query to calculate average readings over a 1-minute window
average_readings_query = """
    SELECT 
        room_id, 
        AVG(temperature) AS avg_temperature, 
        AVG(humidity) AS avg_humidity, 
        window.start AS window_start 
    FROM sensor_table
    GROUP BY room_id, window(timestamp, '1 minute')
"""

# SQL Query to find rooms that need immediate attention based on humidity
attention_needed_query = """
    SELECT 
        room_id, 
        COUNT(*) AS critical_readings 
    FROM sensor_table 
    WHERE humidity < 45 OR humidity > 75
    GROUP BY room_id
"""


### Execute the SQL queries:

Execute each SQL query to create streaming DataFrames.


In [7]:
# Execute the critical temperature query
critical_temperatures_stream = spark.sql(critical_temperature_query)


# Execute the average readings query
average_readings_stream = spark.sql(average_readings_query)

# Execute the attention needed query
attention_needed_stream = spark.sql(attention_needed_query)






### Output the results to the console:

Display the results from each query in real-time.


In [8]:
# Output the results to the console for all queries
critical_query = critical_temperatures_stream.writeStream \
    .outputMode("append") \
    .format("console") \
    .queryName("Critical Temperatures") \
    .start()

average_query = average_readings_stream.writeStream \
    .outputMode("complete") \
    .format("console") \
    .queryName("Average Readings") \
    .start()

attention_query = attention_needed_stream.writeStream \
    .outputMode("complete") \
    .format("console") \
    .queryName("Attention Needed") \
    .start()



-------------------------------------------
Batch: 0
-------------------------------------------
+-------+-----------+--------+---------+
|room_id|temperature|humidity|timestamp|
+-------+-----------+--------+---------+
+-------+-----------+--------+---------+



-------------------------------------------
Batch: 1
-------------------------------------------
+-------+-----------+------------------+--------------------+
|room_id|temperature|          humidity|           timestamp|
+-------+-----------+------------------+--------------------+
|      0|       15.0|45.289661480453205|2024-10-24 19:02:...|
|      0|       15.0| 56.62315503429199|2024-10-24 19:02:...|
|      0|       15.0|48.409065823809854|2024-10-24 19:02:...|
|      0|       15.0| 57.88128462287001|2024-10-24 19:02:...|
+-------+-----------+------------------+--------------------+



-------------------------------------------
Batch: 0
-------------------------------------------
+-------+-----------------+
|room_id|critical_readings|
+-------+-----------------+
+-------+-----------------+



-------------------------------------------
Batch: 2
-------------------------------------------
+-------+-----------+-----------------+--------------------+
|room_id|temperature|         humidity|           timestamp|
+-------+-----------+-----------------+--------------------+
|      0|       15.0|68.02642659202229|2024-10-24 19:02:...|
|      0|       15.0|63.50720181145601|2024-10-24 19:02:...|
|      0|       15.0|60.98749993482241|2024-10-24 19:02:...|
|      0|       15.0|53.83330754335445|2024-10-24 19:02:...|
+-------+-----------+-----------------+--------------------+

-------------------------------------------
Batch: 0
-------------------------------------------
+-------+---------------+------------+------------+
|room_id|avg_temperature|avg_humidity|window_start|
+-------+---------------+------------+------------+
+-------+---------------+------------+------------+



-------------------------------------------
Batch: 3
-------------------------------------------
+-------+-----------+------------------+--------------------+
|room_id|temperature|          humidity|           timestamp|
+-------+-----------+------------------+--------------------+
|      0|       15.0| 53.39721722576067|2024-10-24 19:02:...|
|      0|       15.0| 50.71216075695803|2024-10-24 19:03:...|
|      0|       15.0| 56.71786472845962|2024-10-24 19:03:...|
|      0|       15.0| 47.47342228246402|2024-10-24 19:03:...|
|      0|       15.0| 54.09489515262146|2024-10-24 19:03:...|
|      0|       15.0| 66.38976709305646|2024-10-24 19:02:...|
|      0|       15.0|54.938762017163526|2024-10-24 19:03:...|
|      0|       15.0| 65.84887752627787|2024-10-24 19:03:...|
|      0|       15.0| 53.63921303423464|2024-10-24 19:03:...|
|      0|       15.0| 57.72365309840194|2024-10-24 19:03:...|
|      0|       15.0|  62.1757725230145|2024-10-24 19:02:...|
|      0|       15.0|43.12537979558

...

-------------------------------------------
Batch: 6
-------------------------------------------
+-------+-----------+-----------------+--------------------+
|room_id|temperature|         humidity|           timestamp|
+-------+-----------+-----------------+--------------------+
|      0|       15.0|56.86339875094347|2024-10-24 19:04:...|
|      0|       15.0|43.53462250227166|2024-10-24 19:04:...|
|      0|       15.0|60.41541006698252|2024-10-24 19:04:...|
|      0|       15.0|53.39917012539304|2024-10-24 19:04:...|
|      0|       15.0| 65.2566278306985|2024-10-24 19:04:...|
|      0|       15.0|46.29213490628381|2024-10-24 19:04:...|
+-------+-----------+-----------------+--------------------+

-------------------------------------------
Batch: 2
-------------------------------------------
+-------+------------------+------------------+-------------------+
|room_id|   avg_temperature|      avg_humidity|       window_start|
+-------+------------------+------------------+------------

-------------------------------------------
Batch: 7
-------------------------------------------
+-------+-----------+------------------+--------------------+
|room_id|temperature|          humidity|           timestamp|
+-------+-----------+------------------+--------------------+
|      0|       15.0| 61.96503265934962|2024-10-24 19:04:...|
|      0|       15.0|54.051430060955525|2024-10-24 19:04:...|
|      0|       15.0| 67.03699004965534|2024-10-24 19:04:...|
|      0|       15.0| 40.73044245488607|2024-10-24 19:04:...|
|      0|       15.0|55.091898119464076|2024-10-24 19:04:...|
|      0|       15.0| 58.06092223893379|2024-10-24 19:04:...|
+-------+-----------+------------------+--------------------+

-------------------------------------------
Batch: 3
-------------------------------------------
+-------+-----------------+
|room_id|critical_readings|
+-------+-----------------+
|      7|                4|
|      3|                9|
|      8|               11|
|      0|        

-------------------------------------------
Batch: 8
-------------------------------------------
+-------+-----------+------------------+--------------------+
|room_id|temperature|          humidity|           timestamp|
+-------+-----------+------------------+--------------------+
|      0|       15.0| 67.44470667750521|2024-10-24 19:04:...|
|      0|       15.0|44.770013702942464|2024-10-24 19:04:...|
|      0|       15.0| 40.95063280668987|2024-10-24 19:04:...|
|      0|       15.0| 66.56677876636996|2024-10-24 19:04:...|
|      0|       15.0| 64.91697379069711|2024-10-24 19:04:...|
|      0|       15.0| 66.38216101957347|2024-10-24 19:04:...|
|      0|       15.0| 53.47099699579119|2024-10-24 19:04:...|
+-------+-----------+------------------+--------------------+

-------------------------------------------
Batch: 3
-------------------------------------------
+-------+------------------+------------------+-------------------+
|room_id|   avg_temperature|      avg_humidity|       w

-------------------------------------------
Batch: 9
-------------------------------------------
+-------+-----------+------------------+--------------------+
|room_id|temperature|          humidity|           timestamp|
+-------+-----------+------------------+--------------------+
|      0|       15.0|43.837505470007365|2024-10-24 19:04:...|
|      0|       15.0| 46.57795058313488|2024-10-24 19:04:...|
|      0|       15.0|62.433790933529956|2024-10-24 19:04:...|
|      0|       15.0| 41.55486484171089|2024-10-24 19:04:...|
|      0|       15.0| 46.27429873962451|2024-10-24 19:04:...|
|      0|       15.0| 57.99599723669192|2024-10-24 19:04:...|
+-------+-----------+------------------+--------------------+

-------------------------------------------
Batch: 4
-------------------------------------------
+-------+-----------------+
|room_id|critical_readings|
+-------+-----------------+
|      7|                9|
|      3|               12|
|      8|               12|
|      0|        

[Stage 28:(129 + 8) / 200][Stage 29:>   (0 + 0) / 8][Stage 30:>   (0 + 0) / 8]8]

### Keep the streams running:

Ensure that the streaming queries continue to run to process incoming data.


In [9]:
# Keep the streams running

print("********Critical Temperature Values*******")
critical_query.awaitTermination()

print("********Average Readings Values********")
average_query.awaitTermination()
print("********Attention Needed Values********")
attention_query.awaitTermination()


********Critical Temperature Values*******


-------------------------------------------
Batch: 10
-------------------------------------------
+-------+-----------+------------------+--------------------+
|room_id|temperature|          humidity|           timestamp|
+-------+-----------+------------------+--------------------+
|      0|       15.0| 65.45002057569195|2024-10-24 19:05:...|
|      0|       15.0| 52.27857508086993|2024-10-24 19:05:...|
|      0|       15.0| 56.69796734950124|2024-10-24 19:05:...|
|      0|       15.0|40.773334489151914|2024-10-24 19:04:...|
|      0|       15.0| 56.79281450416834|2024-10-24 19:05:...|
|      0|       15.0| 60.08532654625242|2024-10-24 19:04:...|
|      0|       15.0| 41.92105221107599|2024-10-24 19:05:...|
+-------+-----------+------------------+--------------------+

-------------------------------------------
Batch: 4
-------------------------------------------
+-------+------------------+------------------+-------------------+
|room_id|   avg_temperature|      avg_humidity|       

-------------------------------------------
Batch: 11
-------------------------------------------
+-------+-----------+------------------+--------------------+
|room_id|temperature|          humidity|           timestamp|
+-------+-----------+------------------+--------------------+
|      0|       15.0|54.249225118337236|2024-10-24 19:05:...|
|      0|       15.0| 44.67558032649458|2024-10-24 19:05:...|
|      0|       15.0|45.407567281400695|2024-10-24 19:05:...|
|      0|       15.0| 51.82849024829269|2024-10-24 19:05:...|
|      0|       15.0| 66.22108567146205|2024-10-24 19:05:...|
|      0|       15.0| 56.16031105652783|2024-10-24 19:05:...|
+-------+-----------+------------------+--------------------+

-------------------------------------------
Batch: 5
-------------------------------------------
+-------+-----------------+
|room_id|critical_readings|
+-------+-----------------+
|      7|                9|
|      3|               12|
|      8|               14|
|      0|       

-------------------------------------------
Batch: 12
-------------------------------------------
+-------+-----------+------------------+--------------------+
|room_id|temperature|          humidity|           timestamp|
+-------+-----------+------------------+--------------------+
|      0|       15.0| 62.23471878840921|2024-10-24 19:05:...|
|      0|       15.0| 64.68912130722386|2024-10-24 19:05:...|
|      0|       15.0| 48.35646978549682|2024-10-24 19:05:...|
|      0|       15.0|53.759807389432396|2024-10-24 19:05:...|
|      0|       15.0| 67.65017673656052|2024-10-24 19:05:...|
|      0|       15.0| 45.18501973301548|2024-10-24 19:05:...|
+-------+-----------+------------------+--------------------+

-------------------------------------------
Batch: 5
-------------------------------------------
+-------+------------------+------------------+-------------------+
|room_id|   avg_temperature|      avg_humidity|       window_start|
+-------+------------------+------------------+-

-------------------------------------------
Batch: 13
-------------------------------------------
+-------+-----------+------------------+--------------------+
|room_id|temperature|          humidity|           timestamp|
+-------+-----------+------------------+--------------------+
|      0|       15.0| 54.86097340988664|2024-10-24 19:05:...|
|      0|       15.0| 41.13554786127295|2024-10-24 19:05:...|
|      0|       15.0|54.643087856754434|2024-10-24 19:05:...|
|      0|       15.0|48.202957373327855|2024-10-24 19:05:...|
|      0|       15.0|44.547439050934095|2024-10-24 19:05:...|
|      0|       15.0|63.213043405151055|2024-10-24 19:05:...|
+-------+-----------+------------------+--------------------+

-------------------------------------------
Batch: 6
-------------------------------------------
+-------+-----------------+
|room_id|critical_readings|
+-------+-----------------+
|      7|               11|
|      3|               13|
|      8|               16|
|      0|       

...

[Stage 190:(194 + 6) / 200][Stage 191:=> (5 + 2) / 8][Stage 192:>  (0 + 0) / 8]]

-------------------------------------------
Batch: 64
-------------------------------------------
+-------+-----------+------------------+--------------------+
|room_id|temperature|          humidity|           timestamp|
+-------+-----------+------------------+--------------------+
|      0|       15.0| 48.94752156477691|2024-10-24 19:16:...|
|      0|       15.0| 64.49342486148767|2024-10-24 19:16:...|
|      0|       15.0| 51.75397151440147|2024-10-24 19:16:...|
|      0|       15.0|57.240766905378486|2024-10-24 19:16:...|
|      0|       15.0| 65.80075438249793|2024-10-24 19:16:...|
|      0|       15.0| 44.68721921245895|2024-10-24 19:16:...|
+-------+-----------+------------------+--------------------+



-------------------------------------------
Batch: 31
-------------------------------------------
+-------+------------------+------------------+-------------------+
|room_id|   avg_temperature|      avg_humidity|       window_start|
+-------+------------------+------------------+-------------------+
|      5| 32.81233199277502| 54.91561334957613|2024-10-24 19:10:00|
|      6|31.639365626059085| 54.44605197997148|2024-10-24 19:05:00|
|      0|              15.0| 53.46360529389417|2024-10-24 19:13:00|
|      4|   32.975907224603| 53.73641332015965|2024-10-24 19:05:00|
|      4|31.356650831198696| 55.06248224294551|2024-10-24 19:12:00|
|      3| 31.30334756415192| 54.30858759674779|2024-10-24 19:06:00|
|      9| 33.07751321593638| 54.97661362058853|2024-10-24 19:06:00|
|      6|31.059170160202846| 55.43932652686444|2024-10-24 19:16:00|
|      0|              15.0|56.018438702074654|2024-10-24 19:11:00|
|      2| 32.48513772413735| 55.68460434921325|2024-10-24 19:15:00|
|      4|  31.8953

-------------------------------------------
Batch: 65
-------------------------------------------
+-------+-----------+------------------+--------------------+
|room_id|temperature|          humidity|           timestamp|
+-------+-----------+------------------+--------------------+
|      0|       15.0| 66.74999373533896|2024-10-24 19:16:...|
|      0|       15.0| 60.75284727432387|2024-10-24 19:16:...|
|      0|       15.0| 40.82023216891109|2024-10-24 19:16:...|
|      0|       15.0| 66.36950593120784|2024-10-24 19:16:...|
|      0|       15.0|59.455709736220925|2024-10-24 19:16:...|
|      0|       15.0|47.131926302361926|2024-10-24 19:16:...|
+-------+-----------+------------------+--------------------+

-------------------------------------------
Batch: 32
-------------------------------------------
+-------+-----------------+
|room_id|critical_readings|
+-------+-----------------+
|      7|               68|
|      3|               66|
|      8|               68|
|      0|      

[Stage 196:(29 + 8) / 200][Stage 197:>  (0 + 0) / 8][Stage 198:>  (0 + 0) / 8]]]